# NetCDF read/write
[netCDF](https://www.unidata.ucar.edu/software/netcdf/) is one of the most widely used format in oceanography.
This notebook explains how to read and write data from/in this format.   

Historically, the first Julia module to work with netCDF was https://github.com/JuliaGeo/NetCDF.jl, then another module was developed by [@abarth](https://github.com/Alexander-Barth): [NCDatasets](https://github.com/Alexander-Barth/NCDatasets.jl). This is the one we will use it in this notebook and in the rest of the training material.

In [ ]:
using NCDatasets
using Downloads
include("../config.jl")

## Read a netCDF
We will use the GEBCO bathymetry as an example.

In [ ]:
download_check(gebco16file, gebco16fileURL)

First we open the file for reading (`"r"` option) and can get the equivalent of a `ncdump` command with `@show`:   

In [ ]:
ds = Dataset(gebco16file, "r");
@show ds;

We now know the variable name and attributes, as well as the dimensions.    
Let's read the longitude, latitude and bathymetry.

In [ ]:
lon = ds["lon"];
lat = ds["lat"];
bathymetry = ds["bat"];

Note that with the previous commands, we load both the variable values and their attributes.

In [ ]:
@show typeof(lon);

Then if we want to access the values only, we can use similar commands but with the `[:]` (or `[:,:]` for 2D variables) at the end:

In [ ]:
lonval = ds["lon"][:];
latval = ds["lat"][:];
b = ds["bat"][:, :];

which yield different arrays, for instance:

In [ ]:
@show typeof(b);

⚠️ Don't forget to close the dataset!

In [ ]:
close(ds);

### Ensure the netCDF is closed
A clean way to ensure a file (not only a netCDF) is closed) is to use the *do-block syntax*.      
Previous operations can be re-written as:

In [ ]:
Dataset(gebco16file, "r") do ds
    lonval = ds["lon"][:]
    latval = ds["lat"][:]
    b = ds["bat"][:, :]
end;

and even if an error occurs within the do-block, the file will be closed.

### Making a simple plot
We present the plotting modules in more details in [this notebook](./1-05-plots-maps.ipynb).  
Here we represent the bathymetry on a global map, with a decreased resolution (factor `NN`).

In [ ]:
using Makie, CairoMakie, GeoMakie

In [ ]:
NN = 10
f = Figure()
ax = GeoAxis(f[1, 1], title = "GEBCO bathymetry at a lower resolution", dest = "+proj=moll")
hm = heatmap!(ax, lonval[1:NN:end], latval[1:NN:end], b[1:NN:end, 1:NN:end])
display(f)

## Write a netCDF
### Time series
Let's start with a very simple example: a temperature time series (1-dimensional dataset), corresponding to the mean temperature in Uccle, Belgium              
(source: [IRM](https://www.meteo.be/fr/climat/climat-de-la-belgique/normales-climatiques-a-uccle/temperature/temperature-moyenne)).

In [ ]:
temperatureUccle = [3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18.0, 14.9, 11.1, 6.8, 3.9];

In [ ]:
length(temperatureUccle)

First, let's create the file (clobber mode indicated by `"c"`).    
Note that we need to ensure the file doesn't already exist (otherwise we remove it).

In [ ]:
outputfile = joinpath(datadir, "temperature_belgium.nc")
if isfile(outputfile)
    rm(outputfile)
    @info "Removing file $(outputfile)"
else
    @info "Creating a new netCDF"
end
ds = Dataset(outputfile, "c");

Then we create a time dimension. Its size should match that of the temperature vector.

In [ ]:
defDim(ds, "time", length(temperatureUccle))

Now we can also create the *temperature* variable, which has *time* as dimension.

In [ ]:
temperature = defVar(ds, "temperature", Float64, ("time",));

We will fill this variable with the values previously defined (note again the `[:]`):

In [ ]:
temperature[:] = temperatureUccle;

and don't forget to close the `ds`:

In [ ]:
close(ds);

If you have [`ncdump`](https://www.unidata.ucar.edu/software/netcdf/netcdf-4/newdocs/netcdf/ncdump.html) available on your machine, you can easily check the file content:
```julia
run(`ncdump temperature_series.nc`)
```
Otherwise you can use the first part of the notebook, in which we've shown how to read a netCDF file.

In [ ]:
# command = `ncdump $(outputfile)`
# run(command)

## Exercice
Create a netCDF file storing a 2D field (it can be a random field or real data).     
The procedure is similar to the previous example, except that we will work with longitude and latitude dimensions, instead of time.

The solution is available in [1-03-netCDF-2Dimensions.ipynb](../Exercises/1-03-netCDF-2Dimensions.ipynb).